# Horizontally Federated XGBoost

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

In this tutorial, we will learn how to use SecretFlow to train tree models for horizontal federation. Secretflow provides tree modeling capabilities for horizontal scenarios(SFXgboost), The usage of SFXgboost is similar to XGBoost, you can easily convert your existing XGBoost program into a federated model for SecretFlow.

## Xgboost

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework.

official tutorial [XGBoost tutorials](https://xgboost.readthedocs.io/en/latest/tutorials/index.html).


### prepare secretflow devices 

In [4]:
%load_ext autoreload
%autoreload 2

import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'charlie'], address='local', num_cpus=64)
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The version of SecretFlow: 1.4.0.dev20240105


2024-01-10 08:17:15,825	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 3300233216 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.77gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-01-10 08:17:16,000	INFO worker.py:1538 -- Started a local Ray instance.


### XGBoost Example

In [5]:
import xgboost as xgb
import pandas as pd
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('dermatology'))
df.fillna(value=0)
print(df.dtypes)
y = df['class']
y = y - 1
x = df.drop(columns="class")
dtrain = xgb.DMatrix(x, y)
dtest = dtrain
params = {
    'max_depth': 4,
    'objective': 'multi:softmax',
    'min_child_weight': 1,
    'max_bin': 10,
    'num_class': 6,
    'eval_metric': 'merror',
}
num_round = 4
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evals=watchlist, early_stopping_rounds=2)

erythema                                      int64
scaling                                       int64
definite_borders                              int64
itching                                       int64
koebner_phenomenon                            int64
polygonal_papules                             int64
follicular_papules                            int64
oral_mucosal_involvement                      int64
knee_and_elbow_involvement                    int64
scalp_involvement                             int64
family_history                                int64
melanin_incontinence                          int64
eosinophils_in_the_infiltrate                 int64
pnl_infiltrate                                int64
fibrosis_of_the_papillary_dermis              int64
exocytosis                                    int64
acanthosis                                    int64
hyperkeratosis                                int64
parakeratosis                                 int64
clubbing_of_

### Then, How to do federated xgboost in secretflow?
1. Use federate Binning method based on iteration to calculate the global bucket information combined with the data of all sides, which was used as the candidate to enter the subsequent construction procedure.
2. The data is input into each Client XGBoost engine to calculate G & H.
3. Train federated boosting model  
   1) Data is reassigned to the node to be split.  
   2) The sum of grad and the sum of hess are calculated according to the previously calculated binning buckets.  
   3) Send the sum of grad and the sum of hess to server，server use secure aggregation to produce global summary，then choose best split point，Send best split info back to clients.  
   4) Clients Updates local model.  
4. Finish training，and save model.

Create 3 entities in the Secretflow environment [Alice, Bob, Charlie]. Where `Alice` and `Bob` are clients, and `Charlie` is the server,then you can happily start `Federate Boosting`.

###  Prepare Data

In [6]:
from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation import SecureAggregator
from secretflow.security.compare import SPUComparator
from secretflow.utils.simulation.datasets import load_dermatology

aggr = SecureAggregator(charlie, [alice, bob])
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))
comp = SPUComparator(spu)
data = load_dermatology(parts=[alice, bob], aggregator=aggr, comparator=comp)
data.fillna(value=0, inplace=True)

INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.


### Prepare Params

In [7]:
params = {
    # XGBoost parameter tutorial
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    'max_depth': 4,  # max depth
    'eta': 0.3,  # learning rate
    'objective': 'multi:softmax',  # objection function，support "binary:logistic","reg:logistic","multi:softmax","multi:softprob","reg:squarederror"
    'min_child_weight': 1,  # The minimum value of weight
    'lambda': 0.1,  # L2 regularization term on weights (xgb's lambda)
    'alpha': 0,  # L1 regularization term on weights (xgb's alpha)
    'max_bin': 10,  # Max num of binning
    'num_class': 6,  # Only required in multi-class classification
    'gamma': 0,  # Same to min_impurity_split,The minimux gain for a split
    'subsample': 1.0,  # Subsample rate by rows
    'colsample_bytree': 1.0,  # Feature selection rate by tree
    'colsample_bylevel': 1.0,  # Feature selection rate by level
    'eval_metric': 'merror',  # supported eval metric：
    # 1. rmse
    # 2. rmsle
    # 3. mape
    # 4. logloss
    # 5. error
    # 6. error@t
    # 7. merror
    # 8. mlogloss
    # 9. auc
    # 10. aucpr
    # Special params in SFXgboost
    # Required
    'hess_key': 'hess',  # Required, Mark hess columns, optionally choosing a column name that is not in the data set
    'grad_key': 'grad',  # Required，Mark grad columns, optionally choosing a column name that is not in the data set
    'label_key': 'class',  # Required，ark label columns, optionally choosing a column name that is not in the data set
}

### Create SFXgboost

In [8]:
from secretflow.ml.boost.homo_boost import SFXgboost

bst = SFXgboost(server=charlie, clients=[alice, bob])

INFO:root:Create proxy actor <class 'secretflow.ml.boost.homo_boost.homo_booster_worker.HomoBooster'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.homo_boost.homo_booster_worker.HomoBooster'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.homo_boost.homo_booster_worker.HomoBooster'> with party charlie.


run SFXgboost

In [9]:
bst.train(data, data, params=params, num_boost_round=6)

(_run pid=46292) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=46292) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=46292) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=46292) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=46292) WARNING:jax._src.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=46267) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=46267) INFO:ja

(HomoBooster pid=49791) [0]	train-merror:0.85000	valid-merror:0.85000
(HomoBooster pid=49790) [0]	train-merror:0.00546	valid-merror:0.00546
(HomoBooster pid=49789) [0]	train-merror:0.02186	valid-merror:0.02186


(HomoBooster pid=49790) INFO:root:fit for iter_round=1 done
(HomoBooster pid=49789) INFO:root:fit for iter_round=1 done
(HomoBooster pid=49791) INFO:root:fit for iter_round=1 done


(HomoBooster pid=49791) [1]	train-merror:0.85000	valid-merror:0.85000
(HomoBooster pid=49790) [1]	train-merror:0.00546	valid-merror:0.00546
(HomoBooster pid=49789) [1]	train-merror:0.01093	valid-merror:0.01093


(HomoBooster pid=49790) INFO:root:fit for iter_round=2 done
(HomoBooster pid=49789) INFO:root:fit for iter_round=2 done
(HomoBooster pid=49791) INFO:root:fit for iter_round=2 done


(HomoBooster pid=49791) [2]	train-merror:0.85000	valid-merror:0.85000
(HomoBooster pid=49790) [2]	train-merror:0.00546	valid-merror:0.00546
(HomoBooster pid=49789) [2]	train-merror:0.01093	valid-merror:0.01093


(HomoBooster pid=49790) INFO:root:fit for iter_round=3 done
(HomoBooster pid=49789) INFO:root:fit for iter_round=3 done
(HomoBooster pid=49791) INFO:root:fit for iter_round=3 done


(HomoBooster pid=49791) [3]	train-merror:0.85000	valid-merror:0.85000
(HomoBooster pid=49790) [3]	train-merror:0.00546	valid-merror:0.00546
(HomoBooster pid=49789) [3]	train-merror:0.01639	valid-merror:0.01639


(HomoBooster pid=49790) INFO:root:fit for iter_round=4 done
(HomoBooster pid=49789) INFO:root:fit for iter_round=4 done
(HomoBooster pid=49791) INFO:root:fit for iter_round=4 done


(HomoBooster pid=49791) [4]	train-merror:0.85000	valid-merror:0.85000
(HomoBooster pid=49790) [4]	train-merror:0.00546	valid-merror:0.00546
(HomoBooster pid=49789) [4]	train-merror:0.01093	valid-merror:0.01093


Now our Federated XGBoost training is complete, where the BST is the federated Boost object.

## Conclusion
* This tutorial introduces how to use tree models for training etc.
* SFXgboost encapsulates the logic of the federated subtree model. Sfxgboost trained models remain compatible with XGBoost, and we can directly use the existing infrastructure for online prediction and so on.
* Next, you can try SFXgboost on your data, just need to follow this tutorial.
